---
# Perbandingan Pendekatan Case-Based Reasoning Berbasis TF-IDF: Cosine Similarity vs SVM dalam Analisis Putusan Perdata PMH

Anggota Kelompok :

1. Rofiq Samanhudi - 202210370311260

2. Muhammad Ikbar Ananda Sulistio - 202210370311236

## Tugas Besar Mata Kuliah Penalaran Komputer (B)
---



## Tahap 1 – Membangun Case Base

### Tujuan

Tahap ini bertujuan untuk mengumpulkan dan mempersiapkan korpus dokumen putusan pengadilan dalam bentuk teks yang bersih dan terstruktur. Dokumen ini akan menjadi basis bagi seluruh proses reasoning pada sistem Case-Based Reasoning (CBR).

---

### Langkah Kerja

#### 1. Seleksi dan Pengunduhan Dokumen

- Pemilihan jenis perkara: **Perdata - Perbuatan Melawan Hukum (PMH)**.
- Proses scraping dilakukan dari situs putusan Mahkamah Agung.
- File unduhan disimpan dalam bentuk PDF di direktori: `PMH_PDF/`.

---

#### 2. Konversi dan Ekstraksi Teks

- File PDF dikonversi menjadi teks menggunakan library `pdfminer`.
- Setiap file diproses untuk mengambil seluruh isi putusan sebagai teks mentah.
- Output teks disimpan per dokumen di folder: `data/raw/`.

---

#### 3. Pembersihan Teks (Cleaning)

- Menghapus watermark, header, footer, dan disclaimer resmi dari Mahkamah Agung.
- Normalisasi teks dilakukan dengan:
  - Menghilangkan baris kosong ganda.
  - Mengubah huruf menjadi lowercase.
  - Menghapus spasi berlebih.
- Rasio retensi konten dihitung untuk mengevaluasi hasil pembersihan.

---

#### 4. Validasi dan Logging

- Setiap file yang berhasil diproses dilaporkan ke dalam log:
  - Nomor dokumen
  - Status pembersihan (OK atau rendah)
  - Lokasi penyimpanan
  - Tanggal dan waktu proses
- Log disimpan di file: `logs/cleaning.log`.

---

### Output Tahap Ini

- Total dokumen yang diproses: **70 kasus**.
- Dokumen teks bersih disimpan di: `data/raw/`
- Log proses dan status pembersihan: `logs/cleaning.log`

---

Tahap pertama ini berhasil membentuk korpus awal untuk sistem CBR. Dokumen telah dikonversi ke format teks (>80%) dan dibersihkan secara sistematis, siap untuk tahap representasi selanjutnya.


In [ ]:
import os
import re
import glob
from pdfminer.high_level import extract_text
from datetime import datetime

# === Directory configuration ===
SOURCE_DIR = 'PMH_PDF'
DEST_DIR = 'data/raw'
LOG_DEST = 'logs/cleaning.log'

def purify_text(input_text):
    """Remove unwanted headers, footers, and disclaimers from extracted text."""
    initial_length = len(input_text) if input_text else 1  # Prevent division by zero

    # Eliminate watermarks and page indicators
    processed_text = input_text
    processed_text = processed_text.replace("Direktori Putusan Mahkamah Agung Republik Indonesia", "")
    processed_text = processed_text.replace("putusan.mahkamahagung.go.id", "")
    processed_text = re.sub(r'halaman\s*\d+', '', processed_text, flags=re.IGNORECASE)
    processed_text = processed_text.replace("M a h ka m a h A g u n g R e p u blik In d o n esia\n", "")
    
    # Record length after initial cleaning
    intermediate_length = len(processed_text)
    
    # Remove disclaimer content
    processed_text = processed_text.replace("Disclaimer\n", "")
    processed_text = processed_text.replace(
        "Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n", "")
    processed_text = processed_text.replace(
        "pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu.\n", "")
    processed_text = processed_text.replace(
        "Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n", "")
    processed_text = processed_text.replace(
        "Email : kepaniteraan@mahkamahagung.go.id    Telp : 021-384 3348 (ext.318)\n", "")

    # Final text normalization
    processed_text = processed_text.lower()
    processed_text = ' '.join(processed_text.split())

    # Calculate content retention ratio
    retention_ratio = intermediate_length / initial_length

    return processed_text, retention_ratio

def record_log(case_id, retention_ratio):
    """Log processing details to a file."""
    os.makedirs(os.path.dirname(LOG_DEST), exist_ok=True)
    with open(LOG_DEST, 'a', encoding='utf-8') as log:
        timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
        log.write(f"[{timestamp}] {case_id} | Content Integrity: {retention_ratio:.2%}\n")

def handle_pdf_batch():
    """Process all PDFs in the source directory."""
    os.makedirs(DEST_DIR, exist_ok=True)
    pdf_paths = sorted(glob.glob(os.path.join(SOURCE_DIR, '*.pdf')))
    
    if not pdf_paths:
        print("No PDFs found in the source directory.")
        return

    for index, pdf in enumerate(pdf_paths, 1):
        case_id = f"case_{index:03d}"
        output_path = os.path.join(DEST_DIR, f"{case_id}.txt")

        try:
            # Extract text from PDF
            raw_text = extract_text(pdf)

            # Clean and validate text
            final_text, integrity_ratio = purify_text(raw_text)

            # Check content integrity
            if integrity_ratio < 0.8:
                print(f"[WARNING] {case_id} retains only {integrity_ratio:.2%} of content.")
            else:
                print(f"[OK] {case_id} processed ({integrity_ratio:.2%} valid).")
                # Save cleaned text if valid
                with open(output_path, 'w', encoding='utf-8') as output:
                    output.write(final_text)

            # Log the result
            record_log(case_id, integrity_ratio)

        except Exception as err:
            print(f"[ERROR] Failed to process {pdf}: {str(err)}")

def main():
    """Initiate PDF processing workflow."""
    print("Beginning PDF text extraction...")
    handle_pdf_batch()
    print("PDF processing completed.")

if __name__ == "__main__":
    main()

Beginning PDF text extraction...
[OK] case_001 processed (96.35% valid).
[OK] case_002 processed (96.25% valid).
[OK] case_003 processed (96.36% valid).
[OK] case_004 processed (96.48% valid).
[OK] case_005 processed (96.34% valid).
[OK] case_006 processed (96.38% valid).
[OK] case_007 processed (96.10% valid).
[OK] case_008 processed (96.30% valid).
[OK] case_009 processed (96.43% valid).
[OK] case_010 processed (96.40% valid).
[OK] case_011 processed (96.42% valid).
[OK] case_012 processed (96.45% valid).
[OK] case_013 processed (95.81% valid).
[OK] case_014 processed (96.56% valid).
[OK] case_015 processed (96.24% valid).
[OK] case_016 processed (96.26% valid).
[OK] case_017 processed (96.22% valid).
[OK] case_018 processed (96.21% valid).
[OK] case_019 processed (96.46% valid).
[OK] case_020 processed (96.71% valid).
[OK] case_021 processed (96.55% valid).
[OK] case_022 processed (96.23% valid).
[OK] case_023 processed (96.41% valid).
[OK] case_024 processed (96.18% valid).
[OK] ca

## Tahap 2 – Case Representation

### Tujuan

Tahap ini bertujuan untuk merepresentasikan setiap putusan dalam struktur data yang terorganisir. Representasi ini akan menjadi basis data terstruktur yang siap digunakan untuk proses retrieval dan analisis pada sistem Case-Based Reasoning (CBR).

---

### Langkah Kerja

#### 1. Ekstraksi Metadata

Setiap dokumen hasil pembersihan teks dianalisis untuk mengekstrak informasi penting sebagai metadata menggunakan pendekatan berbasis pola (regex) dan pemrosesan teks. Metadata yang diekstrak meliputi:

- Nomor Perkara (`no_perkara`)
- Tanggal Putusan (`tanggal`)
- Ringkasan Fakta (`ringkasan_fakta`)
- Pasal yang digunakan atau dasar hukum (`pasal`)
- Pihak-pihak yang terlibat (tergugat dan penggugat)
- Isi teks lengkap dari putusan (`text_full`)

---

#### 2. Penyimpanan Data Terstruktur

Hasil ekstraksi metadata disimpan dalam dua format file untuk kemudahan pemrosesan:

- **CSV** → `data/processed/cases_extracted.csv`
- **JSON** → `data/processed/cases_extracted.json`

Struktur data untuk masing-masing kasus mencakup kolom:
- `case_id`
- `no_perkara`
- `tanggal`
- `ringkasan_fakta`
- `pasal`
- `pihak`
- `text_full`

---

#### 3. Feature Engineering

Untuk meningkatkan kualitas representasi kasus, dilakukan proses rekayasa fitur (feature engineering) sebagai berikut:

- **Panjang Teks (Length)**: Menghitung jumlah kata dalam `ringkasan_fakta`.
- **Bag-of-Words (BoW)**: Menghitung frekuensi kata yang muncul dalam ringkasan.
- **QA-Pairs Sederhana**: Menghasilkan pasangan pertanyaan dan jawaban berbasis konten ringkasan, misalnya:
  - Apa nomor perkaranya?
  - Apa pasal yang relevan?
  - Siapa pihak tergugat?
  - Apa ringkasan faktanya?

---

#### 4. Penyimpanan Fitur

Fitur-fitur yang telah direkayasa disimpan dalam file JSON untuk fleksibilitas dalam analisis selanjutnya:

- `data/processed/features_length.json`
- `data/processed/features_bow.json`
- `data/processed/features_qa_pairs.json`

---

Tahap representasi berhasil membentuk struktur data yang terorganisir untuk semua kasus yang tersedia. Masing-masing kasus telah dilengkapi dengan metadata penting dan fitur tambahan untuk mendukung proses retrieval dan reasoning pada tahapan berikutnya dalam sistem CBR.


In [2]:
import os
import re
import json
import csv
from collections import Counter

RAW_FOLDER = 'data/raw'
PROCESSED_FOLDER = 'data/processed'

os.makedirs(PROCESSED_FOLDER, exist_ok=True)

def extract_metadata(text):
    metadata = {}

    # Nomor Perkara
    match = re.search(r'Nomor\s*:\s*(\S+)', text, re.IGNORECASE)
    metadata['no_perkara'] = match.group(1) if match else ''

    # Tanggal Putusan
    match = re.search(r'\b(?:putusan|diputuskan)\s*(?:pada|tanggal)?\s*(\d{1,2}\s+\w+\s+\d{4})', text, re.IGNORECASE)
    metadata['tanggal'] = match.group(1) if match else ''

    # Pasal yang disebutkan
    match = re.findall(r'Pasal\s+\d+\s+[^\n.,;]*', text, re.IGNORECASE)
    metadata['pasal'] = match if match else []

    # Ringkasan Fakta Sederhana
    match = re.search(r'(menimbang\s+bahwa.*?)((?:menimbang|mengingat|memperhatikan)\b.*)', text, re.IGNORECASE | re.DOTALL)
    metadata['ringkasan_fakta'] = match.group(1).strip() if match else ''

    # Terdakwa & Korban (opsional)
    terdakwa = re.findall(r'terdakwa(?:\s*:\s*|\s+)([A-Z][a-zA-Z\s]+)', text)
    korban = re.findall(r'korban(?:\s*:\s*|\s+)([A-Z][a-zA-Z\s]+)', text)
    metadata['pihak'] = {
        'terdakwa': terdakwa[0] if terdakwa else '',
        'korban': korban[0] if korban else ''
    }

    return metadata

def feature_engineering(text, metadata):
    tokens = re.findall(r'\b\w+\b', text.lower())
    length = len(tokens)
    bow = dict(Counter(tokens))

    qa_pairs = {
        "Apa nomor perkaranya?": metadata.get("no_perkara", ""),
        "Apa pasal yang dilanggar?": ", ".join(metadata.get("pasal", [])),
        "Siapa terdakwanya?": metadata.get("pihak", {}).get("terdakwa", ""),
        "Siapa korbannya?": metadata.get("pihak", {}).get("korban", "")
    }

    return length, bow, qa_pairs

def process_all():
    # === Cek folder dan file .txt ===
    if not os.path.exists(RAW_FOLDER):
        raise FileNotFoundError(f"[ERROR] Folder '{RAW_FOLDER}' tidak ditemukan. Harap buat dan isi dengan file .txt.")

    txt_files = sorted([f for f in os.listdir(RAW_FOLDER) if f.endswith('.txt')])
    if not txt_files:
        raise FileNotFoundError(f"[ERROR] Tidak ada file .txt dalam '{RAW_FOLDER}'. Harap isi folder dengan putusan .txt.")

    all_cases = []
    features_length = {}
    features_bow = {}
    features_qa = {}

    for i, fname in enumerate(txt_files, 1):
        path = os.path.join(RAW_FOLDER, fname)
        with open(path, 'r', encoding='utf-8') as file:
            text = file.read().strip()

        if not text:
            print(f"[PERINGATAN] File kosong dilewati: {fname}")
            continue

        case_id = f"case_{i:03d}"
        metadata = extract_metadata(text)
        metadata['case_id'] = case_id
        metadata['text_full'] = text

        length, bow, qa = feature_engineering(text, metadata)

        all_cases.append(metadata)
        features_length[case_id] = length
        features_bow[case_id] = bow
        features_qa[case_id] = qa

    # Simpan CSV
    csv_path = os.path.join(PROCESSED_FOLDER, 'cases_extracted.csv')
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['case_id', 'no_perkara', 'tanggal', 'ringkasan_fakta', 'pasal', 'pihak', 'text_full']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in all_cases:
            writer.writerow(row)

    # Simpan JSON
    json_path = os.path.join(PROCESSED_FOLDER, 'cases_extracted.json')
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(all_cases, f, ensure_ascii=False, indent=2)

    # Simpan fitur
    with open(os.path.join(PROCESSED_FOLDER, 'features_length.json'), 'w', encoding='utf-8') as f:
        json.dump(features_length, f, ensure_ascii=False, indent=2)
    with open(os.path.join(PROCESSED_FOLDER, 'features_bow.json'), 'w', encoding='utf-8') as f:
        json.dump(features_bow, f, ensure_ascii=False, indent=2)
    with open(os.path.join(PROCESSED_FOLDER, 'features_qa_pairs.json'), 'w', encoding='utf-8') as f:
        json.dump(features_qa, f, ensure_ascii=False, indent=2)

    print(f"[SUKSES] {len(all_cases)} kasus berhasil diproses dan disimpan:")
    print("CSV →", csv_path)
    print("JSON →", json_path)
    print("Length →", os.path.join(PROCESSED_FOLDER, 'features_length.json'))
    print("BoW →", os.path.join(PROCESSED_FOLDER, 'features_bow.json'))
    print("QA Pairs →", os.path.join(PROCESSED_FOLDER, 'features_qa_pairs.json'))

if __name__ == "__main__":
    process_all()


[SUKSES] 70 kasus berhasil diproses dan disimpan:
CSV → data/processed\cases_extracted.csv
JSON → data/processed\cases_extracted.json
Length → data/processed\features_length.json
BoW → data/processed\features_bow.json
QA Pairs → data/processed\features_qa_pairs.json


## Tahap 3 – Case Retrieval

### Tujuan

Tahap ini bertujuan untuk menemukan kasus-kasus lama yang paling relevan dan mirip dengan kasus baru (query) yang diajukan. Proses ini merupakan bagian inti dari sistem Case-Based Reasoning (CBR), yang mendukung pencarian preseden hukum berdasarkan kemiripan konten.

---

### Langkah Kerja

#### 1. Representasi Vektor

Setiap ringkasan fakta dari putusan diubah menjadi representasi vektor menggunakan algoritma **TF-IDF (Term Frequency – Inverse Document Frequency)**. Representasi ini mengubah teks menjadi format numerik agar bisa digunakan untuk penghitungan kemiripan dan klasifikasi.

---

#### 2. Splitting Data

Dataset dibagi menjadi dua bagian:
- **Data latih (training set)** sebanyak 80%
- **Data uji (test set)** sebanyak 20%

Splitting ini digunakan untuk pelatihan model klasifikasi berbasis TF-IDF + SVM, serta evaluasi awal performa retrieval.

---

#### 3. Model Retrieval

Dua pendekatan digunakan untuk proses retrieval:

- **a. TF-IDF + Cosine Similarity (CBR Approach)**  
  Menggunakan cosine similarity untuk mengukur kemiripan vektor TF-IDF antara query dan semua kasus dalam basis data. Top-k kasus dengan skor tertinggi dianggap sebagai yang paling relevan.

- **b. TF-IDF + Support Vector Machine (SVM)**  
  Menggunakan pendekatan supervised learning. Model SVM dilatih untuk memetakan ringkasan fakta ke `case_id`. Diberikan sebuah query, model akan memprediksi case_id yang paling sesuai.

---

#### 4. Fungsi Retrieval

Fungsi retrieval disiapkan untuk masing-masing pendekatan:

- `retrieve_cosine(query: str, k: int = 5)`  
  Mengembalikan top-k `case_id` dengan skor cosine tertinggi dari seluruh basis data.

- `retrieve_svm(query: str)`  
  Mengembalikan satu `case_id` yang diprediksi oleh model SVM berdasarkan klasifikasi.

---

#### 5. Pengujian Awal

Sebanyak 10 query uji disiapkan beserta ground-truth case ID. Untuk masing-masing query:
- Dihitung top-5 hasil cosine similarity.
- Diprediksi top-1 hasil dari model SVM.
- Hasilnya dibandingkan dengan ground-truth.

Query uji disimpan di:
- `data/eval/queries.json`

---

### Output

File hasil yang dihasilkan dari tahap ini meliputi:
- Model klasifikasi (SVM): `03_retrieval_model.pkl`
- TF-IDF Vectorizer: `03_vectorizer.pkl`
- Dataset query uji: `data/eval/queries.json`

---

Tahap Case Retrieval telah berhasil diimplementasikan menggunakan dua pendekatan: unsupervised berbasis kemiripan teks dan supervised berbasis klasifikasi. Keduanya siap digunakan untuk evaluasi dan prediksi solusi dalam tahapan berikutnya.


In [3]:
import os
import json
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# === Konfigurasi path ===
CSV_PATH = "data/processed/cases_extracted.csv"
MODEL_PATH = "03_retrieval_model.pkl"
VECTORIZER_PATH = "03_vectorizer.pkl"
QUERY_PATH = "data/eval/queries.json"
os.makedirs("data/eval", exist_ok=True)

# === Muat data CSV ===
df = pd.read_csv(CSV_PATH)
texts = df['ringkasan_fakta'].fillna(df['text_full']).astype(str)
labels = df['case_id'].astype(str)

# === Split data 80:20 ===
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# === TF-IDF + SVM ===
vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1,2))
X_train_vec = vectorizer.fit_transform(X_train)

classifier = LinearSVC()
classifier.fit(X_train_vec, y_train)

# === Simpan model dan vectorizer ===
joblib.dump(classifier, MODEL_PATH)
joblib.dump(vectorizer, VECTORIZER_PATH)

# === Ambil 10 query uji ===
sample_df = df[df['ringkasan_fakta'].notnull() & (df['ringkasan_fakta'].str.strip() != '')]
sample_df = sample_df.sample(n=10, random_state=42)

sample_queries = [
    {
        "query": row["ringkasan_fakta"][:500],
        "ground_truth": row["case_id"]
    }
    for _, row in sample_df.iterrows()
]

with open(QUERY_PATH, "w", encoding="utf-8") as f:
    json.dump(sample_queries, f, indent=2, ensure_ascii=False)

print("[SUKSES] Model disimpan di :", MODEL_PATH)
print("[SUKSES] Vectorizer disimpan di :", VECTORIZER_PATH)
print("[SUKSES] 10 query uji disimpan di :", QUERY_PATH)

[SUKSES] Model disimpan di : 03_retrieval_model.pkl
[SUKSES] Vectorizer disimpan di : 03_vectorizer.pkl
[SUKSES] 10 query uji disimpan di : data/eval/queries.json


### Tahap 4 – Solution Reuse

#### Tujuan

Tahap ini bertujuan untuk menggunakan kembali solusi dari kasus-kasus terdahulu (putusan pengadilan) untuk memprediksi atau merekomendasikan solusi terhadap kasus baru yang serupa. Pendekatan ini merupakan inti dari prinsip *Case-Based Reasoning (CBR)* yang mengandalkan preseden sebagai referensi pemecahan masalah hukum.

---

#### Langkah Kerja

1. **Ekstraksi Solusi**
   - Dari setiap kasus lama yang diretriev (hasil dari tahap 3), diambil bagian **amar putusan** atau bagian akhir dari dokumen sebagai `solusi_text`.
   - Amar putusan disimpan dalam format `{case_id: solusi_text}`.

2. **Algoritma Prediksi Solusi**

   Terdapat dua pendekatan utama:

   - **CBR (TF-IDF + Cosine Similarity)**  
     Sistem mengambil *top-k* kasus paling mirip berdasarkan cosine similarity, lalu menerapkan salah satu dari 3 strategi:

     - `weighted`: memilih solusi dengan total skor similarity tertinggi.
     - `majority`: memilih solusi yang paling sering muncul di antara *top-k*.
     - `hybrid`: kombinasi antara skor similarity dan frekuensi solusi.

   - **Supervised (TF-IDF + SVM)**  
     Sistem memprediksi `case_id` menggunakan klasifikasi SVM, lalu mengambil solusi dari kasus tersebut.

3. **Ringkasan Solusi**
   - Solusi yang dipilih dipotong sepanjang 300 karakter dari awal teks untuk menghasilkan ringkasan yang singkat dan informatif.

4. **Demo Uji Coba**
   - Sistem diuji menggunakan 10 query uji dari `queries.json` yang telah disiapkan pada tahap 3.
   - Fungsi `predict_with_cosine()` dan `predict_with_svm()` dijalankan untuk setiap query.

---

#### Fungsi Utama

- `predict_with_cosine(query: str, k=5, mode="weighted")`  
  Melakukan pencarian top-k kasus dengan cosine similarity dan menghasilkan solusi berdasarkan:
  - `"weighted"`: solusi dengan total skor similarity tertinggi.
  - `"majority"`: solusi terbanyak di antara hasil retrieval.
  - `"hybrid"`: kombinasi bobot similarity dan jumlah kemunculan.

- `predict_with_svm(query: str)`  
  Menggunakan model klasifikasi SVM untuk memprediksi `case_id`, lalu mengambil solusi dari dokumen tersebut.

---

#### Output

Dua file CSV utama dihasilkan:

- `data/results/predictions_cosine.csv`  
  Berisi hasil prediksi solusi berdasarkan pendekatan CBR Cosine Similarity (mode: `weighted` secara default).

- `data/results/predictions_svm.csv`  
  Berisi hasil prediksi solusi berdasarkan klasifikasi SVM.

Kolom-kolom pada file CSV:

- `query_id`: Nomor urut query.
- `query`: Teks ringkasan kasus baru.
- `predicted_solution`: Solusi yang diprediksi dari dokumen mirip.
- `top_5_case_ids`: Daftar case ID referensi (Cosine).
- `top_1_case_id`: Case ID prediksi tunggal (SVM).

---

#### Catatan

- Pendekatan **CBR (Cosine)** bersifat fleksibel, tidak membutuhkan pelatihan ulang, dan cocok digunakan untuk kasus baru yang belum pernah terlihat.
- Pendekatan **SVM** cocok untuk klasifikasi cepat pada data berlabel, tetapi perlu pelatihan awal dan lebih kaku terhadap variasi baru.
- Mode `hybrid` dapat menjadi alternatif untuk menggabungkan akurasi skor dan kestabilan voting.
- Mode cosine dapat disesuaikan dengan parameter `mode` saat pemanggilan `predict_with_cosine()`.


In [4]:
import os
import json
import joblib
import pandas as pd
from collections import defaultdict, Counter
from sklearn.metrics.pairwise import cosine_similarity

# === PATH KONFIGURASI ===
CASE_DATA = "data/processed/cases_extracted.csv"
QUERY_PATH = "data/eval/queries.json"
MODEL_PATH = "03_retrieval_model.pkl"
VECTORIZER_PATH = "03_vectorizer.pkl"
COSINE_OUTPUT = "data/results/predictions_cosine.csv"
SVM_OUTPUT = "data/results/predictions_svm.csv"
os.makedirs("data/results", exist_ok=True)

# === Load data dan model ===
df = pd.read_csv(CASE_DATA)
with open(QUERY_PATH, encoding="utf-8") as f:
    queries = json.load(f)

classifier = joblib.load(MODEL_PATH)
vectorizer = joblib.load(VECTORIZER_PATH)

# === Siapkan korpus dokumen dan solusi ===
texts = df["ringkasan_fakta"].fillna(df["text_full"]).astype(str).tolist()
case_ids = df["case_id"].astype(str).tolist()
solutions = df["text_full"].astype(str).tolist()

tfidf_matrix = vectorizer.transform(texts)
case_solutions = dict(zip(case_ids, solutions))
case_id_to_text = dict(zip(case_ids, texts))

# === COSINE SIMILARITY DENGAN MODE: majority / weighted / hybrid ===
def predict_with_cosine(query: str, k: int = 5, mode: str = "weighted"):
    query_vec = vectorizer.transform([query])
    scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_k_idx = scores.argsort()[::-1][:k]
    
    solution_scores = defaultdict(float)
    solution_counts = Counter()
    
    for i in top_k_idx:
        cid = case_ids[i]
        score = float(scores[i])
        solution = case_solutions.get(cid, "")[:300]

        if mode == "weighted":
            solution_scores[solution] += score
        elif mode == "majority":
            solution_counts[solution] += 1
        elif mode == "hybrid":
            solution_scores[solution] += score * 0.7 + solution_counts[solution] * 0.3
        else:
            raise ValueError(f"Mode tidak dikenali: {mode}")

    # Pilih solusi terbaik
    if mode in ("weighted", "hybrid"):
        predicted_solution = max(solution_scores.items(), key=lambda x: x[1])[0]
    else:  # majority
        predicted_solution = solution_counts.most_common(1)[0][0]

    top_ids = [case_ids[i] for i in top_k_idx]
    return predicted_solution, top_ids

# === SVM CLASSIFICATION ===
def predict_with_svm(query: str):
    query_vec = vectorizer.transform([query])
    pred_case_id = classifier.predict(query_vec)[0]

    sol = case_solutions.get(str(pred_case_id), "")
    return sol[:300], [pred_case_id]

# === Jalankan untuk semua query ===
cosine_results = []
svm_results = []

for i, q in enumerate(queries, 1):
    query_text = q["query"]

    # COSINE - mode: "weighted" | "majority" | "hybrid"
    cosine_pred, cosine_ids = predict_with_cosine(query_text, mode="weighted")
    cosine_results.append({
        "query_id": i,
        "query": query_text,
        "predicted_solution": cosine_pred,
        "top_5_case_ids": cosine_ids
    })

    # SVM
    svm_pred, svm_id = predict_with_svm(query_text)
    svm_results.append({
        "query_id": i,
        "query": query_text,
        "predicted_solution": svm_pred,
        "top_1_case_id": svm_id[0]
    })

# === Simpan hasil ke CSV ===
pd.DataFrame(cosine_results).to_csv(COSINE_OUTPUT, index=False)
pd.DataFrame(svm_results).to_csv(SVM_OUTPUT, index=False)

print("[SUKSES] Hasil prediksi COSINE disimpan di:", COSINE_OUTPUT)
print("[SUKSES] Hasil prediksi SVM disimpan di   :", SVM_OUTPUT)

[SUKSES] Hasil prediksi COSINE disimpan di: data/results/predictions_cosine.csv
[SUKSES] Hasil prediksi SVM disimpan di   : data/results/predictions_svm.csv


### Tahap 5 – Model Evaluation

#### Tujuan

Tahapan ini bertujuan untuk mengukur dan menganalisis performa sistem Case-Based Reasoning (CBR), baik dalam proses _retrieval_ (pencarian kasus serupa) maupun _prediction_ (penggunaan solusi dari kasus lama). Evaluasi dilakukan untuk menentukan pendekatan mana yang paling efektif: **TF-IDF + Cosine Similarity** atau **TF-IDF + SVM**.

---

#### Langkah Kerja

##### 1. Evaluasi Retrieval

- Evaluasi dilakukan terhadap hasil top-k retrieval dari pendekatan **TF-IDF + Cosine Similarity**.
- Untuk setiap query, dicek apakah `ground_truth` termasuk dalam `top_k` hasil retrieval.
- Metrik evaluasi yang digunakan:
  - **Accuracy**
  - **Precision**
  - **Recall**
  - **F1-Score**
- Perhitungan dilakukan menggunakan fungsi dari `sklearn.metrics`.

##### 2. Evaluasi Prediksi

- Evaluasi dilakukan terhadap hasil prediksi top-1 dari pendekatan **TF-IDF + SVM**.
- Hanya satu `case_id` diprediksi untuk setiap query, lalu dibandingkan dengan `ground_truth`.
- Metrik yang digunakan sama dengan evaluasi retrieval:
  - **Accuracy**
  - **Precision**
  - **Recall**
  - **F1-Score**

##### 3. Visualisasi & Laporan

- Visualisasi hasil evaluasi dalam bentuk **grafik batang** (_bar chart_) yang membandingkan performa model.
- Dilakukan **error analysis** terhadap query yang salah diprediksi, dan disimpan dalam file JSON.

---

#### Implementasi

- Fungsi evaluasi:
  - `eval_retrieval()`: untuk evaluasi top-k retrieval berdasarkan cosine similarity.
  - `eval_prediction()`: untuk evaluasi top-1 prediksi menggunakan SVM.
- Fungsi tambahan:
  - `save_errors()`: menyimpan query yang gagal diprediksi dengan benar.
- Evaluasi dilakukan berdasarkan file hasil prediksi:
  - `data/results/predictions_cosine.csv`
  - `data/results/predictions_svm.csv`
- File ground truth:
  - `data/eval/queries.json`

---

#### Output

- **Hasil Evaluasi**
  - `data/eval/retrieval_metrics.csv`  
    Berisi metrik evaluasi untuk TF-IDF + Cosine Similarity.
  - `data/eval/prediction_metrics.csv`  
    Berisi metrik evaluasi untuk TF-IDF + SVM.

- **Visualisasi Performa**
  - `data/eval/performance_comparison.png`  
    Grafik perbandingan metrik antar model.

- **Error Analysis**
  - `data/eval/error_cases_cosine.json`  
    Daftar query yang gagal ditemukan dengan benar oleh pendekatan cosine similarity.
  - `data/eval/error_cases_svm.json`  
    Daftar query yang gagal diprediksi oleh pendekatan SVM.

---

Tahap ini menjadi penentu efektivitas sistem CBR dalam konteks hukum perdata (PMH), dan menjadi dasar pemilihan model untuk deployment dan pengembangan lanjutan.


In [5]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from typing import Dict

# === PATH SETUP ===
COSINE_PRED = "data/results/predictions_cosine.csv"
SVM_PRED = "data/results/predictions_svm.csv"
EVAL_QUERIES = "data/eval/queries.json"
RETRIEVAL_METRIC_OUTPUT = "data/eval/retrieval_metrics.csv"
PREDICTION_METRIC_OUTPUT = "data/eval/prediction_metrics.csv"
ERROR_COSINE_OUTPUT = "data/eval/error_cases_cosine.json"
ERROR_SVM_OUTPUT = "data/eval/error_cases_svm.json"
os.makedirs("data/eval", exist_ok=True)

# === LOAD GROUND TRUTH ===
with open(EVAL_QUERIES, encoding='utf-8') as f:
    ground_truth_data = json.load(f)
    gt_dict = {i + 1: str(item["ground_truth"]).strip() for i, item in enumerate(ground_truth_data)}

# === EVALUASI RETRIEVAL COSINE (TOP-K) ===
def eval_retrieval(pred_file: str, model_name: str, k: int = 5) -> Dict:
    df = pd.read_csv(pred_file)
    y_true = []
    y_pred = []

    for _, row in df.iterrows():
        query_id = int(row["query_id"])
        gt = str(gt_dict.get(query_id, "")).strip()

        try:
            pred_ids = eval(row["top_5_case_ids"])
            if not isinstance(pred_ids, list):
                pred_ids = []
            pred_ids = [str(i).strip() for i in pred_ids]
        except Exception:
            pred_ids = []

        hit = gt in pred_ids[:k]
        y_true.append(1)
        y_pred.append(1 if hit else 0)

    return {
        "model": model_name,
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1_score": f1_score(y_true, y_pred, zero_division=0)
    }

# === EVALUASI SVM PREDICTION (TOP-1) ===
def eval_prediction(pred_file: str, model_name: str) -> Dict:
    df = pd.read_csv(pred_file)
    y_true, y_pred = [], []

    for _, row in df.iterrows():
        query_id = int(row["query_id"])
        gt = str(gt_dict.get(query_id, "")).strip()
        pred = str(row.get("top_1_case_id", "")).strip()

        if not pred:
            continue

        hit = pred == gt
        y_true.append(1)
        y_pred.append(1 if hit else 0)

    if not y_true:
        print(f"[PERINGATAN] Tidak ada prediksi valid untuk model {model_name}")
        return {
            "model": model_name,
            "accuracy": 0.0,
            "precision": 0.0,
            "recall": 0.0,
            "f1_score": 0.0
        }

    return {
        "model": model_name,
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1_score": f1_score(y_true, y_pred, zero_division=0)
    }

# === SIMPAN KASUS GAGAL ===
def save_errors(pred_file: str, output_file: str, top_k: int = 5, mode: str = 'cosine'):
    df = pd.read_csv(pred_file)
    error_cases = []

    for _, row in df.iterrows():
        query_id = int(row["query_id"])
        gt = str(gt_dict.get(query_id, "")).strip()

        if mode == 'cosine':
            try:
                pred_ids = eval(row["top_5_case_ids"])
                if not isinstance(pred_ids, list):
                    pred_ids = []
                pred_ids = [str(i).strip() for i in pred_ids]
            except Exception:
                pred_ids = []
            hit = gt in pred_ids[:top_k]
        else:
            pred_ids = [str(row.get("top_1_case_id", "")).strip()]
            hit = gt == pred_ids[0]

        if not hit:
            error_cases.append({
                "query_id": query_id,
                "query": row["query"],
                "predicted": pred_ids,
                "ground_truth": gt
            })

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(error_cases, f, indent=2, ensure_ascii=False)

# === EKSEKUSI & SIMPAN HASIL ===
retrieval_metrics = eval_retrieval(COSINE_PRED, "TF-IDF + Cosine")
prediction_metrics = eval_prediction(SVM_PRED, "TF-IDF + SVM")

pd.DataFrame([retrieval_metrics]).to_csv(RETRIEVAL_METRIC_OUTPUT, index=False)
pd.DataFrame([prediction_metrics]).to_csv(PREDICTION_METRIC_OUTPUT, index=False)

save_errors(COSINE_PRED, ERROR_COSINE_OUTPUT, mode='cosine')
save_errors(SVM_PRED, ERROR_SVM_OUTPUT, mode='svm')

# === VISUALISASI ===
combined_df = pd.DataFrame([retrieval_metrics, prediction_metrics])
if not combined_df.empty:
    combined_df.set_index("model")[["accuracy", "precision", "recall", "f1_score"]].plot(
        kind="bar", figsize=(10, 6), title="Perbandingan Model"
    )
    plt.ylabel("Score")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig("data/eval/performance_comparison.png")
    plt.close()

print("[SUKSES] Tahap 5 - Evaluasi Model selesai")
print(f"- Hasil evaluasi retrieval disimpan di: {RETRIEVAL_METRIC_OUTPUT}")
print(f"- Hasil evaluasi prediksi disimpan di : {PREDICTION_METRIC_OUTPUT}")
print(f"- Visualisasi disimpan di             : data/eval/performance_comparison.png")
print(f"- Error cosine disimpan di           : {ERROR_COSINE_OUTPUT}")
print(f"- Error svm disimpan di              : {ERROR_SVM_OUTPUT}")


[SUKSES] Tahap 5 - Evaluasi Model selesai
- Hasil evaluasi retrieval disimpan di: data/eval/retrieval_metrics.csv
- Hasil evaluasi prediksi disimpan di : data/eval/prediction_metrics.csv
- Visualisasi disimpan di             : data/eval/performance_comparison.png
- Error cosine disimpan di           : data/eval/error_cases_cosine.json
- Error svm disimpan di              : data/eval/error_cases_svm.json


### Kesimpulan

---

### Penutup